In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import os
import shutil
from pathlib import Path
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import classification_report,confusion_matrix
import itertools
from sklearn.model_selection import train_test_split 
from sklearn.prepr ocessing import normalize

df = pd.read_csv('/floyd/input/csv/images.csv', header=None)
dir = r'/floyd/input/images_copy'

img_list = os.listdir(dir)
img_list = [Path(img).stem for img in img_list]
df = df[df[0].isin(img_list)]


df['price'] = df[5]
arr = df[2].unique() 
df['color'] = df[2].transform(lambda x: np.asscalar(np.array(np.where(arr == x))))
df['style'] = df[4]
df['nyco_class'] = df[3]
df['sku'] = df[0]
df['sales'] = df[1]
df['above500'] = df[1] >= 500

df['nyco_class'].unique()
df['style'].unique()

Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


array([ 611, 7565,  506, ..., 1888, 5349, 4376])

In [2]:
X = np.column_stack((df['price'].values, df['style'].values, df['nyco_class'].values, df['color'].values))
y = df['above500'].values.reshape(-1, 1)

# X = normalize(X, axis=0, norm='max')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1) 
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1) 


print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

print(np.sum(y_train == 1))
print(np.sum(y_test == 1))
print(np.sum(y_val == 1))

(9423, 1)
(524, 1)
(523, 1)
3407
174
198


In [3]:
classifier = Sequential()
classifier.add(Dense(output_dim = 300, init = 'uniform', activation = 'relu', input_dim = 4))
classifier.add(Dense(output_dim = 500, init = 'uniform', activation = 'relu'))
classifier.add(Dense(output_dim = 500, init = 'uniform', activation = 'relu'))
classifier.add(Dense(output_dim = 500, init = 'uniform', activation = 'relu'))
classifier.add(Dense(output_dim = 200, init = 'uniform', activation = 'relu'))
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=4, units=300, kernel_initializer="uniform")`
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=500, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=500, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=500, kernel_initializer="uniform")`
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense`

In [4]:
classifier.fit(X_train, y_train, batch_size = 128, epochs = 1000, verbose=0, validation_data= (X_val, y_val))

In [5]:
Y_pred = classifier.predict(X_test)
y_pred = np.round(Y_pred) 
print(classification_report(y_test, y_pred)) #
print(np.sum(y_pred))
print(y_pred.shape[0])
print(np.sum(y_pred)/y_pred.shape[0])
print(np.sum(y_test))

t = lambda x: 1 if x>0.9 else 0
y_pred = np.array([t(x) for x in Y_pred])
print(classification_report(y_test, y_pred)) #
print(np.sum(y_pred))
print(y_pred.shape[0])
print(np.sum(y_pred)/y_pred.shape[0])
print(np.sum(y_test))

              precision    recall  f1-score   support

       False       0.81      0.90      0.85       350
        True       0.74      0.57      0.65       174

    accuracy                           0.79       524
   macro avg       0.77      0.74      0.75       524
weighted avg       0.78      0.79      0.78       524

136.0
524
0.2595419847328244
174
              precision    recall  f1-score   support

       False       0.71      0.98      0.82       350
        True       0.79      0.18      0.29       174

    accuracy                           0.71       524
   macro avg       0.75      0.58      0.56       524
weighted avg       0.73      0.71      0.64       524

39
524
0.07442748091603053
174


In [6]:
from random import randint
r = str(randint(100, 999))

classifier.save('sequential-data'+r+'.h5')

In [7]:
# 10000 epochs
#        precision    recall  f1-score   support

#        False       0.82      0.82      0.82       350
#         True       0.64      0.64      0.64       174

#     accuracy                           0.76       524
#    macro avg       0.73      0.73      0.73       524
# weighted avg       0.76      0.76      0.76       524

In [8]:
# 2000 epochs
#  precision    recall  f1-score   support

#        False       0.82      0.82      0.82       350
#         True       0.64      0.64      0.64       174

#     accuracy                           0.76       524
#    macro avg       0.73      0.73      0.73       524
# weighted avg       0.76      0.76      0.76       524

In [9]:
# 1000 epochs
#  precision    recall  f1-score   support

#        False       0.86      0.81      0.84       350
#         True       0.66      0.74      0.70       174

#     accuracy                           0.79       524
#    macro avg       0.76      0.78      0.77       524
# weighted avg       0.80      0.79      0.79       524

In [10]:
# 1500 epochs
# precision    recall  f1-score   support

#        False       0.79      0.84      0.81       350
#         True       0.63      0.55      0.59       174

#     accuracy                           0.74       524
#    macro avg       0.71      0.69      0.70       524
# weighted avg       0.74      0.74      0.74       524
